In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import random
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
import matplotlib.font_manager as fm

[(f.name, f.fname) for f in fm.fontManager.ttflist if 'Nanum' in f.name]
plt.rcParams['font.family'] = 'Malgun Gothic'

In [4]:
df_01 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_01_DEMO.csv')
df_02 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_02_PDDE.csv')
df_03 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_03_COP_U.csv')
df_04 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_04_PD_CLAC.csv')
df_05 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_05_BR.csv')
df_06 = pd.read_csv('../dataset/LPOINT_BIG_COMP/LPOINT_BIG_COMP_06_LPAY.csv')

In [5]:
df_02 = df_02[df_02['chnl_dv'] == 1]

In [6]:
df_02.drop(columns=['buy_am', 'de_dt', 'rct_no', 'chnl_dv'], axis=1, inplace=True)

In [7]:
df_merge_02_04 = pd.merge(df_02, df_04, on='pd_c')
df_merge_02_04 

,cust,cop_c,br_c,pd_c,de_hr,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm
0,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의
1,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의
2,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의
3,M430112881,A01,A010039,PD0290,10,1,남성티셔츠,남성의류,남성의류상의
4,M506355556,A01,A010032,PD0290,15,1,남성티셔츠,남성의류,남성의류상의
...,...,...,...,...,...,...,...,...,...
3953237,M033482606,A05,A050003,PD0075,20,1,혈당계/혈압계,건강용품,건강측정용품
3953238,M489266563,A05,A050426,PD1221,15,1,청소기/광택기,자동차용품,세차용품
3953239,M296090374,A05,A050260,PD0543,11,1,이발기,생활/주방가전,이미용가전
3953240,M560002381,A05,A050039,PD0543,12,1,이발기,생활/주방가전,이미용가전


### 상품 상위 5개 추출

In [7]:
def frequency_product(cust, df_merge_02_04):
    person_id = cust
    pd_c = df_merge_02_04[df_merge_02_04['cust'] == person_id][['clac_hlv_nm', 'buy_ct']]
    pd_c = pd_c.groupby('clac_hlv_nm')['buy_ct'].sum().reset_index()
    pd_c = pd_c.sort_values(by='buy_ct', ascending=False, ignore_index=True)
    freq_product = []
    for i in range(5):
        try:
            freq_product.append(pd_c['clac_hlv_nm'][i])

        except: KeyError
            
    return freq_product

In [8]:
frequency_product('M430112881', df_merge_02_04)

['채소', '냉장식품', '수산물', '유제품', '음료']

In [ ]:
df_01['freq_product'] = df_01.apply(lambda x : frequency_product(x['cust'], df_merge_02_04), axis=1)

### 평점 변환

In [8]:
def convert_rating(cust, clac_hlv_nm, df):
    
    person_id = cust    
    try:
        pd_c = df[df['cust'] == cust][['clac_hlv_nm', 'buy_ct']]
        pd_c = pd_c.groupby('clac_hlv_nm')['buy_ct'].sum().reset_index()
        pd_c = pd_c.sort_values(by='buy_ct', ascending=False, ignore_index=True)

        pd_c['buy_ct'] = (pd_c['buy_ct'] - pd_c['buy_ct'].min()) / (pd_c['buy_ct'].max() - pd_c['buy_ct'].min())
        pd_c['buy_ct'][0] = random.uniform(0.85, 1.01) 
        pd_c['buy_ct'] = np.round(pd_c['buy_ct'], 2) * 10

        res = float(pd_c[pd_c['clac_hlv_nm'] == clac_hlv_nm]['buy_ct'])
        
    except: ValueError
        
    return res

In [11]:
res = convert_rating(df_merge_02_04['cust'][10], df_merge_02_04['clac_hlv_nm'][10], df_merge_02_04)
res

3.8

In [ ]:
df_merge_02_04['ratings'] = df_merge_02_04.apply(lambda x : convert_rating(x['cust'], x['clac_hlv_nm'], df_merge_02_04), axis=1)

### 멀티프로세싱

In [18]:
import multiprocessing as mp

num_cores = mp.cpu_count()

def parallel_dataframe(df, func):
    global num_cores
    df_split = np.array_split(df, num_cores)
    pool = mp.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    
    return df

In [19]:
tqdm.pandas()
def make_ratings(data):
    data['ratings'] = data.progress_apply(lambda x : convert_rating(x['cust'], x['clac_hlv_nm'], data), axis=1)
    return data

In [21]:
df = parallel_dataframe(df_merge_02_04, make_ratings)

100%|████████████████████████████████| 658874/658874 [10:28:50<00:00, 17.46it/s]


In [22]:
df.to_csv('df_merge_02_04_ratings.csv')

In [24]:
df['ratings'].isna().sum()

13256

In [27]:
df_test = df[df['ratings'].isna()]
df_test

,cust,cop_c,br_c,pd_c,de_hr,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm,ratings
282,M260927455,A01,A010021,PD0290,15,1,남성티셔츠,남성의류,남성의류상의,NaN
1342,M403286666,A01,A010024,PD0290,16,1,남성티셔츠,남성의류,남성의류상의,NaN
1346,M403286666,A01,A010024,PD0290,17,1,남성티셔츠,남성의류,남성의류상의,NaN
2519,M560023501,A01,A010002,PD0290,10,1,남성티셔츠,남성의류,남성의류상의,NaN
3040,M614062136,A01,A010045,PD0290,14,1,남성티셔츠,남성의류,남성의류상의,NaN
...,...,...,...,...,...,...,...,...,...,...
3952965,M081494913,A05,A050130,PD0552,11,1,전기프라이팬,생활/주방가전,주방가전,NaN
3952974,M784453498,A05,A050238,PD0990,17,1,TV장식장/거치대,영상/음향가전,TV,NaN
3953035,M224574767,A05,A050274,PD1004,19,1,컴팩트,영상/음향가전,카메라/캠코더,NaN
3953093,M518148689,A05,A050267,PD0998,15,1,CDP/카세트,영상/음향가전,오디오,NaN


In [43]:
df.dropna(inplace=True)

In [44]:
df_others = parallel_dataframe(df_test, make_ratings)
df_others

100%|██████████████████████████████████████| 2210/2210 [00:10<00:00, 211.69it/s]


,cust,cop_c,br_c,pd_c,de_hr,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm,ratings
282,M260927455,A01,A010021,PD0290,15,1,남성티셔츠,남성의류,남성의류상의,9.1
1342,M403286666,A01,A010024,PD0290,16,1,남성티셔츠,남성의류,남성의류상의,8.6
1346,M403286666,A01,A010024,PD0290,17,1,남성티셔츠,남성의류,남성의류상의,9.5
2519,M560023501,A01,A010002,PD0290,10,1,남성티셔츠,남성의류,남성의류상의,9.2
3040,M614062136,A01,A010045,PD0290,14,1,남성티셔츠,남성의류,남성의류상의,9.7
...,...,...,...,...,...,...,...,...,...,...
3952965,M081494913,A05,A050130,PD0552,11,1,전기프라이팬,생활/주방가전,주방가전,9.8
3952974,M784453498,A05,A050238,PD0990,17,1,TV장식장/거치대,영상/음향가전,TV,8.8
3953035,M224574767,A05,A050274,PD1004,19,1,컴팩트,영상/음향가전,카메라/캠코더,9.3
3953093,M518148689,A05,A050267,PD0998,15,1,CDP/카세트,영상/음향가전,오디오,9.1
